In [30]:
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml
import tarfile

In [31]:
print(gc.RUN_NAME)
job = 'ungrib_press'
file_types = ['press']
real = False

LIST_S_LINKS = [
    'ungrib',
    'ungrib.exe',
    'link_grib.csh',
]

LIST_H_LINKS = [
    'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [32]:
con = sq.connect(gc.PATH_DB)

In [52]:
importlib.reload(un)
run_row = un.get_run_row()
print(run_row)

job_row = un.get_next_row(job=job)
print(job_row)
un.update_run_table(val=job_row[job]+1,
                    job=job,
                    date=job_row['date']
                   )
job_path = un.getmk_job_path(run_row,job_row,job)
print(job_path)

untar_path = os.path.join(job_path,'untar')

conf_path = un.get_conf_path(run_row)
print(conf_path)

type_rows = pd.DataFrame([un.get_type_row(ft,job_row) for ft in file_types])
print(type_rows)

name_list = un.skim_namelist_copy(
    conf_path,job_path,date =job_row.date,prefix=job
)
print(name_list)

if gc.ID=='taito_login':
    un.copy_hard_links(conf_path,job_path,LIST_H_LINKS)
    un.copy_soft_links(gc.PATH_WPS,job_path,LIST_S_LINKS)
    importlib.reload(un)
    un.untar_the_files(type_rows,joba_path)

index                             1
run_name             run_2019_02_20
metgrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object
date            2017-12-05 00:00:00
ungrib_surf                       0
ungrib_press                      0
ungrib_lake                       0
Name: 0, dtype: object
/tmp/wrf_management/data_folder/runs/run_2019_02_20/2017_12_05/ungrib_press
/Users/diego/wrf_management/wrf_management/config_dir/run_2019_02_20
                  date  i  downloaded                      name  untarred  \
0  2017-12-05 00:00:00  4           1  cdas1.20171205.pgrbh.tar         0   

    type  
0  press  
OrderedDict([('share', Namelist([('wrf_core', 'ARW'), ('max_dom', 4), ('start_date', ['2017-12-05_00:00:00', '2017-12-05_00:00:00', '2017-12-05_00:00:00', '2017-12-05_00:00:00']), ('end_date', ['2017-12-05_18:00:00', '2017-12-05_18:00:00', '2017-12-05_18:00:00', '2017-12-05_18:00:00']), ('interval_seconds',

In [17]:
r = type_rows.iloc[0]
type_ = r.type
source_tar_path = gc.FILE_TYPES[type_]['data_tar']
source_tar_path = os.path.join(
    gc.PATH_DATA,
    source_tar_path,
    r['name']
)

In [20]:
tf=tarfile.TarFile(source_tar_path)

In [22]:
tf.getmembers()

[<TarInfo 'cdas1.t00z.pgrbh01.grib2' at 0x1c1dbf62a0>,
 <TarInfo 'cdas1.t00z.pgrbh02.grib2' at 0x1c1def3818>,
 <TarInfo 'cdas1.t00z.pgrbh03.grib2' at 0x1c1def3750>,
 <TarInfo 'cdas1.t00z.pgrbh04.grib2' at 0x1c1def38e0>,
 <TarInfo 'cdas1.t00z.pgrbh05.grib2' at 0x1c1def39a8>,
 <TarInfo 'cdas1.t00z.pgrbh06.grib2' at 0x1c1def3a70>,
 <TarInfo 'cdas1.t00z.pgrbh07.grib2' at 0x1c1def3b38>,
 <TarInfo 'cdas1.t00z.pgrbh08.grib2' at 0x1c1def3c00>,
 <TarInfo 'cdas1.t00z.pgrbh09.grib2' at 0x1c1def3cc8>,
 <TarInfo 'cdas1.t00z.pgrbhanl.grib2' at 0x1c1def3d90>,
 <TarInfo 'cdas1.t06z.pgrbh01.grib2' at 0x1c1def3e58>,
 <TarInfo 'cdas1.t06z.pgrbh02.grib2' at 0x1c1def3f20>,
 <TarInfo 'cdas1.t06z.pgrbh03.grib2' at 0x1c1e00d048>,
 <TarInfo 'cdas1.t06z.pgrbh04.grib2' at 0x1c1e00d110>,
 <TarInfo 'cdas1.t06z.pgrbh05.grib2' at 0x1c1e00d1d8>,
 <TarInfo 'cdas1.t06z.pgrbh06.grib2' at 0x1c1e00d2a0>,
 <TarInfo 'cdas1.t06z.pgrbh07.grib2' at 0x1c1e00d368>,
 <TarInfo 'cdas1.t06z.pgrbh08.grib2' at 0x1c1e00d4f8>,
 <TarInfo

In [29]:
importlib.reload(un)
date_formated=un.date_file_format(job_row.date)

In [34]:
job_row

date            2017-12-03 00:00:00
ungrib_surf                       0
ungrib_press                      0
ungrib_lake                       0
Name: 0, dtype: object

In [42]:
val = job_row[job]+1
print(val)
sql = """
update {tb}
set {job} = {val}
where date('{dt}')=date(date)
""".format(dt=job_row.date,tb=gc.RUN_NAME,val=val,job=job)
print(sql)

1

update run_2019_02_20
set ungrib_press = 1
where date('2017-12-03 00:00:00')=date(date)



In [45]:
con = sq.connect(gc.PATH_DB)
try: 
    cu = con.cursor()
    cu.execute(sql)
    con.commit()
finally: 
    con.close()
